<a href="https://colab.research.google.com/github/Amsterdam-Internships/Readability-Lexical-Simplification/blob/master/Generating_Dutch_Sentences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy==3.2.0
!pip install python-docx 

In [ ]:
import docx
import glob
import spacy 
import nltk

import os
from docx import Document

     |████████████████████████████████| 5.6 MB 13.8 MB/s 
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184507 sha256=b4724df34c1440c0305a072e34fbfffda1970e6c1b53bd77dc351d123c26621e
  Stored in directory: /root/.cache/pip/wheels/f6/6f/b9/d798122a8b55b74ad30b5f52b01482169b445fbb84a11797a6
Successfully built python-docx


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!python -m spacy download nl_core_news_sm

In [ ]:
nlp = spacy.load("nl_core_news_sm")

In [ ]:
with open ("/content/moeilijke woorden.txt","r") as infile:
  data = infile.readlines()

complex_words = []

for line in data:
  if line.startswith("["):
    continue
  if line.count(" ")>0:
    continue
  line = line.strip()
  if len(line) <2:
    continue
  complex_words.append(line)

print(len(complex_words ))

488


In [ ]:
all_text = []

path = '/content/drive/MyDrive/Thesis/notebooks/Preprocessing/Data for Iva'
for filename in glob.glob(os.path.join(path, '*.docx')):
  #  with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
  doc = Document(filename)
  for para in doc.paragraphs:
    text = para.text.strip().replace("\n"," ")
    all_text.append(text)

In [ ]:
print(len(all_text))

10675


In [ ]:
# https://stackoverflow.com/questions/67500193/cleaning-lemmatizing-dutch-dataset-using-spacy
def lemmatizer(texts):
    texts = [text.replace("\n", "").strip() for text in texts]
    docs = nlp.pipe(texts)
    cleaned_lemmas = [[t.lemma_ for t in doc] for doc in docs]

    return cleaned_lemmas

In [ ]:
def lemmatizer(texts):
    texts = [text.replace("\n", "").strip() for text in texts]
    docs = nlp.pipe(texts)
    cleaned_lemmas = [[t.lemma_ for t in doc] for doc in docs]

    return cleaned_lemmas
text = ['duurzame']
lemmatizer(text)

[['duurzaam']]

In [ ]:
complex_sents = []
corresponding_cwords = []

for paragraph in all_text:

  sentences = nltk.sent_tokenize(paragraph)
  paragraph_lemmatized = lemmatizer(sentences)

  for sent, lem_sent in zip(sentences, paragraph_lemmatized):
    sent = sent.replace("\t","")

    for complex_lemma in complex_lemmas:
      if len(complex_lemma): 
        if complex_lemma[0] in lem_sent:
          complex_sents.append(sent)
          corresponding_cwords.append(complex_lemma)

In [ ]:
with open ("/content/dutch_sentences.txt", "w", encoding="utf-8") as outfile:
  for sent, word in zip(complex_sents, corresponding_cwords):
    outfile.write(f"{sent}\t{word[0]}\tannotations\n")